In [6]:
test_lat = 34.05
test_lon = -118.25
test_date = "2022-07-15"

In [8]:
import urllib.request
import urllib.parse
import json

API_KEY = "T7KA7J54DPZKSY4E53M5MWQ9R"

lat = 34.05
lon = -118.25
date_str = "2022-07-15"  # ← ovo mora biti u URL-u

# ✅ Ispravni URL: lokacija + datum
url = (
    f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    f"{urllib.parse.quote(f'{lat},{lon}')}/{date_str}"  # ← datum dodan ovdje!
    f"?key={API_KEY}"
    f"&unitGroup=metric"
    f"&include=days"
    f"&elements=datetime,temp,precip,windspeed,visibility,conditions"
    f"&contentType=json"
)

print("🔍 Ispravni URL:")
print(url)
print("\n📡 Pozivam API...")

try:
    with urllib.request.urlopen(url, timeout=10) as response:
        if response.status == 200:
            data = json.loads(response.read().decode('utf-8'))
            print("\n✅ USPJESNO! Historijski podaci:")
            day = data["days"][0]
            print(f"Datum: {day['datetime']}")  # sad bi trebao biti 2022-07-15
            print(f"Temperatura: {day['temp']}°C")
            print(f"Padaline: {day['precip']} mm")
            print(f"Vjetar: {day['windspeed']} km/h")
            print(f"Vidljivost: {day['visibility']} km")
            print(f"UVJETI: {day['conditions']}")
        else:
            print(f"\n❌ HTTP {response.status}")
except Exception as e:
    print(f"\n❌ Greška: {e}")

🔍 Ispravni URL:
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/34.05%2C-118.25/2022-07-15?key=T7KA7J54DPZKSY4E53M5MWQ9R&unitGroup=metric&include=days&elements=datetime,temp,precip,windspeed,visibility,conditions&contentType=json

📡 Pozivam API...

✅ USPJESNO! Historijski podaci:
Datum: 2022-07-15
Temperatura: 20.6°C
Padaline: 0.0 mm
Vjetar: 13.9 km/h
Vidljivost: 15.1 km
UVJETI: Clear


In [9]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import urllib.request
import urllib.parse
import json

# ---------- 1. API postavke ----------
API_KEY = "T7KA7J54DPZKSY4E53M5MWQ9R"
BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

def get_weather_vr(lat, lon, date_str):
    """Dohvati historijsko vrijeme za jednu točku i datum."""
    url = (
        f"{BASE_URL}{urllib.parse.quote(f'{lat},{lon}')}/{date_str}"
        f"?key={API_KEY}"
        f"&unitGroup=metric"
        f"&include=days"
        f"&elements=datetime,temp,precip,windspeed,visibility,conditions"
        f"&contentType=json"
    )
    try:
        with urllib.request.urlopen(url, timeout=15) as response:
            if response.status == 200:
                data = json.loads(response.read().decode('utf-8'))
                if "days" in data and len(data["days"]) > 0:
                    day = data["days"][0]
                    return {
                        "temp_c": day.get("temp"),
                        "precip_mm": day.get("precip"),
                        "windspeed_kmh": day.get("windspeed"),
                        "visibility_km": day.get("visibility"),
                        "weather_condition": day.get("conditions")
                    }
    except Exception as e:
        # print(f"Greška za {lat},{lon},{date_str}: {e}")
        pass
    return None

# ---------- 2. Mapiranje uvjeta u rizik ----------
def condition_to_risk(cond):
    if pd.isna(cond) or cond is None:
        return 0
    cond = cond.lower()
    if any(kw in cond for kw in ["clear", "partly cloudy", "partially cloudy", "cloudy", "overcast"]):
        return 0
    elif any(kw in cond for kw in ["mist", "light drizzle", "light rain", "drizzle"]):
        return 1
    elif any(kw in cond for kw in ["rain", "snow", "ice", "freezing", "fog"]):
        return 2
    elif any(kw in cond for kw in ["heavy rain", "thunderstorm", "hail", "snowstorm"]):
        return 3
    else:
        return 0

# ---------- 3. Učitaj FARS podatke (Kalifornija, 2015–2023) ----------
project_root = Path(os.getcwd()).parents[0]
data_folder = project_root / 'data' / 'data_accidents'

all_ca = []
for year in range(2015, 2024):
    acc_file = data_folder / str(year) / "accident.csv"
    if not acc_file.exists():
        continue
    try:
        df = pd.read_csv(acc_file, low_memory=False, encoding="utf-8")
    except:
        df = pd.read_csv(acc_file, low_memory=False, encoding="latin1")
    if "STATE" in df.columns:
        df = df[df["STATE"] == 6]  # CA = 6
        df = df[["ST_CASE", "LATITUDE", "LONGITUD", "YEAR", "MONTH", "DAY"]]
        all_ca.append(df)

full_ca = pd.concat(all_ca, ignore_index=True)
print(f"Ukupno nesreća u CA: {len(full_ca):,}")

# ---------- 4. Uzmi 800 nasumičnih nesreća ----------
sampled_ca = full_ca.sample(n=800, random_state=42).copy()

# Čišćenje datuma
sampled_ca = sampled_ca[
    (sampled_ca["LATITUDE"].notna()) &
    (sampled_ca["LONGITUD"].notna()) &
    (sampled_ca["MONTH"].between(1, 12)) &
    (sampled_ca["DAY"].between(1, 31))
]
sampled_ca["date"] = pd.to_datetime(sampled_ca[["YEAR", "MONTH", "DAY"]], errors="coerce")
sampled_ca = sampled_ca[sampled_ca["date"].notna()]
sampled_ca["date_str"] = sampled_ca["date"].dt.strftime("%Y-%m-%d")

print(f"Uzorak: {len(sampled_ca)} nesreća")

# ---------- 5. Dohvati vrijeme za svaku nesreću ----------
weather_data = []
for i, row in tqdm(sampled_ca.iterrows(), total=len(sampled_ca), desc="Dohvaćanje vremena"):
    w = get_weather_vr(row["LATITUDE"], row["LONGITUD"], row["date_str"])
    if w:
        weather_data.append({
            "index": i,
            "temp_c": w["temp_c"],
            "precip_mm": w["precip_mm"],
            "windspeed_kmh": w["windspeed_kmh"],
            "visibility_km": w["visibility_km"],
            "weather_condition": w["weather_condition"]
        })

# Pretvori u DataFrame i spoji
weather_df = pd.DataFrame(weather_data).set_index("index")
final_df = sampled_ca.join(weather_df)

# ---------- 6. Dodaj rizik ----------
final_df["weather_risk_score"] = final_df["weather_condition"].apply(condition_to_risk)

# ---------- 7. Spremi ----------
output = "fars_ca_2015_2023_800sample_visualcrossing.csv"
final_df.to_csv(output, index=False)

print(f"\n✅ Spremljeno: {output}")
print(f"Uspješno dohvaćeno vremena za: {len(weather_df)} / {len(sampled_ca)} nesreća")
print("\nDistribucija rizika:")
print(final_df["weather_risk_score"].value_counts().sort_index())
print("\nPrimjer uvjeta:")
print(final_df["weather_condition"].value_counts().head())

Ukupno nesreća u CA: 32,958
Uzorak: 800 nesreća


Dohvaćanje vremena: 100%|██████████| 800/800 [08:34<00:00,  1.55it/s]


✅ Spremljeno: fars_ca_2015_2023_800sample_visualcrossing.csv
Uspješno dohvaćeno vremena za: 799 / 800 nesreća

Distribucija rizika:
weather_risk_score
0    791
2      9
Name: count, dtype: int64

Primjer uvjeta:
weather_condition
Clear                     430
Partially cloudy          243
Rain, Partially cloudy     92
Rain, Overcast             20
Rain                        8
Name: count, dtype: int64


In [10]:
df = pd.read_csv("fars_ca_2015_2023_800sample_visualcrossing.csv")
df["is_adverse"] = df["weather_risk_score"] >= 2

monthly = df.groupby("MONTH")["is_adverse"].mean()
print(monthly.sort_values(ascending=False))

MONTH
12    0.034483
3     0.017857
1     0.017544
11    0.014925
10    0.013699
7     0.013158
9     0.013158
2     0.000000
4     0.000000
5     0.000000
6     0.000000
8     0.000000
Name: is_adverse, dtype: float64


In [12]:
import pandas as pd

df = pd.read_csv("fars_ca_2015_2023_800sample_visualcrossing.csv")

# Pretvori risk_score u kategorije
df["weather_category"] = df["weather_risk_score"].map({
    0: "Normalno vrijeme",
    1: "Umjereno loše",
    2: "Loše vrijeme",
    3: "Ekstremno vrijeme"
})

print("Broj nesreća po vremenskim uvjetima:")
print(df["weather_category"].value_counts())

Broj nesreća po vremenskim uvjetima:
weather_category
Normalno vrijeme    791
Loše vrijeme          9
Name: count, dtype: int64


--> U Kaliforniji (2015–2023), većina smrtnih prometnih nesreća (98.9%) dogodila se u normalnim vremenskim uvjetima :(